# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf

# Settings

In [3]:
# if True, then it doesn't check if result files already exist and runs everything again
FORCE_RUN = True

In [4]:
PREDICTION_METHOD = "Gene-based"

In [5]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [6]:
# OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
# display(OUTPUT_DATA_DIR)
# OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
INPUT_DATA_DIR = Path(
    OUTPUT_DIR,
    "data",
    "raw",
)
display(INPUT_DATA_DIR)
INPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw')

In [8]:
OUTPUT_PREDICTIONS_DIR = Path(OUTPUT_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions')

# Load PharmacotherapyDB gold standard

In [9]:
gold_standard = pd.read_pickle(
    Path(OUTPUT_DIR, "gold_standard.pkl"),
)

In [10]:
display(gold_standard.shape)

(998, 3)

In [11]:
display(gold_standard.head())

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [12]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load LINCS

## Raw data

In [13]:
input_file = Path(INPUT_DATA_DIR, "lincs-data.pkl").resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw/lincs-data.pkl')

In [14]:
lincs_projection = pd.read_pickle(input_file)

In [15]:
display(lincs_projection.shape)

(7120, 1170)

In [16]:
display(lincs_projection.head())

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
ENSG00000196839,-1.001,-1.835,1.391,1.132,0.257,1.932,0.508,1.408,0.777,0.032,...,-1.692,-0.516,-1.435,-0.317,-0.012,0.641,-0.230,-0.518,-0.177,2.146
ENSG00000170558,1.146,-1.863,0.011,-1.020,1.143,-0.115,1.327,0.310,-1.853,0.872,...,0.354,0.498,0.268,-1.084,-0.142,-0.077,0.633,-1.807,0.032,0.135
ENSG00000117020,-0.693,1.694,-0.804,-0.164,1.145,-1.465,1.221,-0.747,0.829,-0.961,...,-1.196,-0.230,-1.049,-0.347,0.586,0.865,-0.021,2.180,-0.956,0.105
ENSG00000133997,-0.037,0.383,0.269,-0.997,0.185,-0.536,0.424,-0.119,-1.313,0.579,...,-0.343,0.116,-0.245,-0.127,-1.367,0.149,0.117,2.084,1.178,0.772
ENSG00000101473,0.162,-0.899,0.105,-0.090,-1.291,1.404,0.185,0.157,-0.327,-0.026,...,-0.136,-1.115,-0.280,0.200,0.638,-0.197,-0.360,-2.302,-0.117,-0.167


# Load S-PrediXcan

In [17]:
from entity import Trait

In [18]:
phenomexcan_input_file_list = [
    f
    for f in INPUT_DATA_DIR.glob("*.pkl")
    if f.name.startswith("spredixcan-")
    #     if f.name.startswith(("smultixcan-", "spredixcan-"))
]

In [19]:
display(len(phenomexcan_input_file_list))

1

In [21]:
pd.read_pickle(phenomexcan_input_file_list[0]).head()

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
ENSG00000000419,-0.521616,-0.354356,-0.647688,1.110194,-1.786116,1.714687,0.999024,-1.662682,1.576522,-1.402812,...,1.614588,0.954165,-1.325516,-0.290283,1.009580,-1.237774,-1.083244,1.711170,-1.144057,-0.803475
ENSG00000000457,-1.885895,-2.056025,-0.693315,1.342271,1.631252,-1.902825,-1.485516,-2.356658,3.181447,2.094172,...,2.582993,2.004654,2.918376,1.886111,1.242073,1.184227,1.190373,0.773194,-2.302876,1.702888
ENSG00000000460,-1.269773,-2.088193,-2.868509,1.653372,1.307943,-2.114711,1.162425,-1.418749,2.363683,-1.480930,...,1.577850,1.156873,-1.480989,-1.384810,1.820841,1.872488,-1.279276,-1.054766,2.876039,-2.125799
ENSG00000000938,1.521316,-1.344366,-2.155603,1.313485,1.537092,4.348633,-0.604390,-1.816184,-1.049993,1.669054,...,1.369958,-0.446664,-3.131358,1.459237,1.007082,2.096814,0.932845,-2.452785,1.710248,1.730114
ENSG00000000971,-1.409875,-1.325785,-1.674708,1.316074,-1.145304,-0.788946,1.549621,1.224362,-0.670558,-1.915256,...,-1.882554,2.691985,-3.261818,2.756464,-1.038795,-0.576466,-3.605774,1.423109,-1.578721,-1.200703


# Predict drug-disease associations

In [22]:
from drug_disease import (
    #     predict_dotprod,
    predict_dotprod_neg,
    #     predict_pearson,
    #     predict_pearson_neg,
    #     predict_spearman,
    #     predict_spearman_neg,
)

In [23]:
methods_to_run = [predict_dotprod_neg]

In [24]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)

    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)

    # get common genes with lincs
    common_genes = phenomexcan_projection.index.intersection(lincs_projection.index)
    phenomexcan_projection = phenomexcan_projection.loc[common_genes]
    lincs_projection = lincs_projection.loc[common_genes]

    print(f"  shape: {phenomexcan_projection.shape}")

    for prediction_method in methods_to_run:
        for ntc in (None, 50, 100, 250, 500):
            prediction_method(
                lincs_projection,
                phenomexcan_input_file,
                phenomexcan_projection,
                OUTPUT_PREDICTIONS_DIR,
                PREDICTION_METHOD,
                doids_in_gold_standard,
                FORCE_RUN,
                n_top_conditions=ntc,
                use_abs=True,  # because we want positive and negative z-scores from S-PrediXcan
            )

            print("\n")

spredixcan-most_signif-data.pkl
  shape: (7086, 4091)
  predicting all_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,48.857712
std,320.014221
min,-10320.613263
25%,-56.458472
50%,36.735054
75%,132.484432
max,15344.861226


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-data-all_genes-prediction_scores.h5


  predicting top_50_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,8.663541
std,69.670928
min,-3411.490683
25%,-10.723551
50%,6.557789
75%,24.049320
max,8775.102557


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-data-top_50_genes-prediction_scores.h5


  predicting top_100_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


...

    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-data-top_100_genes-prediction_scores.h5


  predicting top_250_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,17.280056
std,117.555031
min,-4802.424684
25%,-19.866577
50%,12.908558
75%,46.174251
max,9082.186853


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-data-top_250_genes-prediction_scores.h5


  predicting top_500_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,22.083991
std,148.249251
min,-5349.318288
25%,-25.438721
50%,16.764418
75%,59.752494
max,9373.322484


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-most_signif-data-top_500_genes-prediction_scores.h5


